# GLACIS Demo: Cryptographic Attestation for AI

**What is GLACIS?** A library that creates tamper-proof receipts for AI operations. 
Prove what your AI did, what data it saw — without sensitive data leaving your environment.

## What You'll Learn
1. **Offline Mode** — Self-signed attestations, no API key needed
2. **Online Mode** — Server-witnessed attestations with Merkle proofs
3. **OpenAI Integration** — Automatic attestation for API calls (optional)

## Core Concept: "Hash Locally, Prove Globally"
- Input/output data is hashed locally (SHA-256)
- Only hashes are transmitted, never actual payloads
- Receipts are cryptographically signed (Ed25519)

## Setup

Install glacis if needed: `pip install glacis`

In [ ]:
import os
from glacis import Glacis

# Generate a 32-byte signing seed for Ed25519
# In production, persist this securely to maintain the same identity
SIGNING_SEED = os.urandom(32)
print(f"Generated signing seed: {SIGNING_SEED.hex()[:16]}...")

## 1. Offline Mode (No API Key Required)

Perfect for development and testing. Receipts are self-signed and stored locally.
Witness status will be `UNVERIFIED` until submitted to the transparency log.

In [ ]:
# Create an offline Glacis client
glacis_offline = Glacis(mode="offline", signing_seed=SIGNING_SEED)
print(f"Mode: {glacis_offline.mode}")

In [ ]:
# Simulate an AI interaction
prompt = "What is the capital of France?"
response = "The capital of France is Paris."

# Create an attestation
# Input and output are hashed locally, never sent anywhere
receipt = glacis_offline.attest(
    service_id="demo-app",
    operation_type="inference",
    input={"prompt": prompt},
    output={"response": response},
    metadata={"model": "gpt-4", "temperature": 0.7},
)

print("Attestation created!")
print(f"  Receipt ID: {receipt.attestation_id}")
print(f"  Payload Hash: {receipt.payload_hash}")
print(f"  Signature: {receipt.signature[:40]}...")
print(f"  Witness Status: {receipt.witness_status}")

In [ ]:
# Verify the receipt
result = glacis_offline.verify(receipt)

print("Verification result:")
print(f"  Signature Valid: {result.signature_valid}")
print(f"  Overall Valid: {result.valid}")
print(f"  Witness Status: {result.witness_status}")

# Receipts are stored locally at ~/.glacis/receipts.db
print(f"\nReceipts stored in: ~/.glacis/receipts.db")

## 2. Online Mode (API Key Required)

For production use. Attestations are witnessed by the Glacis server and included in a Merkle tree.
This provides third-party verifiability and tamper-evident logs.

**Get your API key at https://glacis.io**

In [ ]:
# Set your API key (or set GLACIS_API_KEY environment variable)
GLACIS_API_KEY = os.environ.get("GLACIS_API_KEY", "")

if not GLACIS_API_KEY:
    print("No GLACIS_API_KEY found. Skipping online mode demo.")
    print("Set GLACIS_API_KEY environment variable or get a key at https://glacis.io")
    glacis_online = None
else:
    glacis_online = Glacis(api_key=GLACIS_API_KEY)
    print(f"Mode: {glacis_online.mode}")
    print("Online client ready!")

In [ ]:
if glacis_online:
    # Create a witnessed attestation
    online_receipt = glacis_online.attest(
        service_id="demo-app",
        operation_type="inference",
        input={"prompt": "What is 2 + 2?"},
        output={"response": "4"},
    )
    
    print("Online attestation created!")
    print(f"  Receipt ID: {online_receipt.attestation_id}")
    print(f"  Leaf Index: {online_receipt.leaf_index}")  # Position in Merkle tree
    print(f"  Badge URL: {online_receipt.badge_url}")    # Shareable verification link
    print(f"  Witness Status: WITNESSED")
else:
    print("Skipped - no API key")

In [ ]:
if glacis_online:
    # Verify with Merkle proof
    online_result = glacis_online.verify(online_receipt)
    
    print("Online verification result:")
    print(f"  Signature Valid: {online_result.signature_valid}")
    print(f"  Proof Valid: {online_result.proof_valid}")  # Merkle inclusion proof
    print(f"  Overall Valid: {online_result.valid}")
else:
    print("Skipped - no API key")

## 3. OpenAI Integration (Optional)

Automatically attest every OpenAI API call with a drop-in wrapper.

**Requires:** `pip install glacis[openai]` and `OPENAI_API_KEY`

In [ ]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")

try:
    from glacis.integrations.openai import attested_openai, get_last_receipt
    openai_available = True
except ImportError:
    openai_available = False
    print("OpenAI integration not installed. Run: pip install glacis[openai]")

if openai_available and not OPENAI_API_KEY:
    print("No OPENAI_API_KEY found. Skipping OpenAI demo.")
    openai_client = None
elif openai_available:
    # Create an attested OpenAI client (offline mode)
    openai_client = attested_openai(
        openai_api_key=OPENAI_API_KEY,
        offline=True,
        signing_seed=SIGNING_SEED,
        service_id="openai-demo",
    )
    print("Attested OpenAI client ready!")
else:
    openai_client = None

In [ ]:
if openai_client:
    # Make an API call - attestation happens automatically
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "What is 2 + 2? Answer in one word."}],
        max_tokens=10,
    )
    
    print(f"Response: {response.choices[0].message.content}")
    
    # Get the attestation receipt
    openai_receipt = get_last_receipt()
    print(f"\nAttestation created automatically!")
    print(f"  Receipt ID: {openai_receipt.attestation_id}")
    print(f"  Payload Hash: {openai_receipt.payload_hash}")
else:
    print("Skipped - OpenAI not available")

## Summary

| Mode | API Key | Witness Status | Use Case |
|------|---------|----------------|----------|
| **Offline** | Not needed | `UNVERIFIED` | Development, testing |
| **Online** | Required | `WITNESSED` | Production, compliance |

### What's in a Receipt?

- **attestation_id**: Unique identifier (`oatt_xxx` offline, `att_xxx` online)
- **payload_hash**: SHA-256 of canonical JSON (input + output)
- **signature**: Ed25519 signature over the attestation
- **timestamp**: When the attestation was created
- **leaf_index**: Position in Merkle tree (online only)
- **badge_url**: Shareable verification link (online only)

### Next Steps

1. **Get an API key** at https://glacis.io for production use
2. **Explore the Anthropic integration**: `from glacis.integrations.anthropic import attested_anthropic`
3. **Query the transparency log**: `glacis.query_log(org_id="...", service_id="...")`
4. **Check stored receipts**: `~/.glacis/receipts.db` (SQLite)

In [ ]:
# Cleanup
glacis_offline.close()
if glacis_online:
    glacis_online.close()
print("Done!")